<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [5]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [6]:
def calc_results(model, train_scaled, val_scaled, test_scaled, test_target):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbmgs.best_score_ 
    result_dict['best_index'] = lgbmgs.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['precision'] = precision_score(test_target, y_predict)
    result_dict['recall'] = recall_score(test_target, y_predict)
    result_dict['f1score'] = f1_score(test_target, y_predict)
    result_dict['roc'] = roc_auc_score(test_target, y_predict)
    cm = confusion_matrix(test_target, y_predict)
    result_dict['tn'] = cm[0,0]
    result_dict['fp'] = cm[0,1]
    result_dict['fn'] = cm[1,0]
    result_dict['tp'] = cm[1,1]
    return result_dict

In [7]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [8]:
fname = f'df_mobis_sel.pkl'
directory_for_ml = '../data/data_for_ml/'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name) 

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -4]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -4]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
val_scaled = ss.transform(val_input)
test_scaled = ss.transform(test_input)

In [9]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')

In [10]:
# params_test= { 
#     "num_iterations": [100, 200, 300, 400, 500],
#     "num_class": [1]
#     "metric" : "rmse",
#     "bagging_frequency" : 5,
#     "bagging_seed" : 2018,
#     "verbosity" : -1,

#     # Selected rounded-off params
#     'bagging_fraction': 0.7,
#     'feature_fraction': 0.1,
#     'lambda_l1': 1,
#     'lambda_l2': 0,
#     'max_depth': 9,
#     'min_child_weight': 5,
#     'min_split_gain': 0,
#     'num_leaves': 24 }

In [11]:
params_o = {
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
    "num_iterations" : [100], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.007, 0.008, 0.009, 0.01, 0.015, 0.02], 
    "max_depth": [None, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [None, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [ 0.25, 0.3, 0.35, 0.4, 0.45], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [None, 3, 4, 5],
    "max_delta_step": [0.5, 0.6, 0.7, 0.8, 0.9], # best value found, default = 0
    "scale_pos_weight": [2], # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
    
# ************* 아래 하나씩 테스트해서 취사 선택해야 함. **************
#     "bagging_frequency" : [5], 
#     "early_stopping_rounds" : [200],
#     "num_threads": [0], # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "min_data_in_leaf": [None], # 과적합 방지 파라미터
#     "lambda_l1": [0], # default 0
#     "lambda_l2": [0], #default 0
#     "min_gain_to_split": [None], # 분기하기 위해 필요한 최소한의 gain을 의미, 정규화시 사용
#     "num_threads": = 0, # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "max_bin": [32],  # data 준비할 때 사용하는 parameter     
#     "is_unbalance": [1],  # 불균형 셋 조정
}

In [12]:
def new_num_iter(num_i):
    if num_i <= 10:
        new_num = [10, 50, 100]
    elif num_i <= 100:
        new_num = [50, 100, 150, 200]
    elif num_i <= 200:
        new_num = [150, 200, 300]
    else:
        new_num = [num_i - 100, num_i, num_i + 100]
    return new_num

def new_lr(lr):
    
    def count_zero(x):
        if x < 1:
            iter = 1
            while True:
                if x * 10**iter >= 1:
                    break
                iter = iter+1
            return iter
        elif x >= 1:
            iter = 1
            while True:
                if x / 10**iter < 1:
                    break
                iter = iter+1
            return -iter
    
    def back_step(lr, lr_step):
        if lr - lr_step*2 <=0 :
            tenth = 10**(count_zero(lr)+1)
            return int(lr*tenth/3)/tenth, int(lr*tenth/2)/tenth
        else:
            return lr_step, lr_step*2
    
    if lr <= 0.001:
        lr_step = 0.0002
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 0.01:
        lr_step = 0.002
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 0.1:
        lr_step = 0.02
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 1:
        lr_step = 0.2
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 10:
        lr_step = 2
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    else:
        lr_step = 20
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)

    return [lr-lr_step_m2, lr-lr_step_m1, lr, lr+lr_step, lr+lr_step*2]

def new_depth(depth):
    if depth == None:
        new_depth = [None, 1, 2, 3, 4]
    elif depth <= 2:
        new_depth = [None, 1, 2, 3, 4]
    elif depth <= 10:
        new_depth = [depth-2, depth-1, depth, depth+1, depth+2]
    elif depth <= 20:
        new_depth = [depth-4, depth-2, depth, depth+2, depth+4]
    else:
        new_depth = [depth-8, depth-4, depth, depth+4, depth+8]
    return new_depth

def new_leaves(leaves):
    if leaves == None:
        new_leaves = [2, 3, 4]
    elif leaves <= 3:
        new_leaves = [2, 3, 4]
    elif leaves <= 10:
        new_leaves = [leaves-2, leaves-1, leaves, leaves+1, leaves+2]
    elif leaves <= 20:
        new_leaves = [leaves-4, leaves-2, leaves, leaves+2, leaves+4]
    else:
        new_leaves = [leaves-8, leaves-4, leaves, leaves+4, leaves+8]
    return new_leaves

def new_subsample(subsam):
    if subsam == None:
        return [None, 0.001, 0.002, 0.003]
    elif subsam <= 0.002:
        return [None, 0.001, 0.002, 0.003]
    elif subsam <= 0.01:
        step = 0.001
    elif subsam <= 0.1:
        step = 0.01
    elif subsam <= 1:
        step = 0.2
        
    return [subsam-step, subsam, subsam+step]
    
def make_new_parameter(params_o, df_concat):
    param_new = {}
    param_new["boosting_type"] = ['gbdt']
    
    num_i = df_concat.loc["num_iterations"].iloc[0]
    param_new["num_iterations"] = new_num_iter(num_i) #새로운 값을 중간에 배치, 100단위로 순서
    
    lr = df_concat.loc["learning_rate"].iloc[0]
    param_new["learning_rate"] = new_lr(lr)
    
    depth = df_concat.loc["max_depth"].iloc[0]
    param_new["max_depth"] = new_depth(depth) # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    
    leaves = df_concat.loc["num_leaves"].iloc[0]  
    param_new["num_leaves"] = new_leaves(leaves) #  # 두번째로 중요, num_leaves = 2^max_depth보다 작아야 함. 데이터가 적으면 작은 숫자로
    
    subsam = df_concat.loc["subsample"].iloc[0]  
    param_new["subsample"] = new_subsample(subsam) # 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    
    param_new["colsample_bytree"] = [0.25, 0.3, 0.35, 0.4, 0.45] # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    param_new["objective"] = ['binary']
    param_new["metric"] = ['binary_logloss'] # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    param_new["n_estimators"] = [None, 3, 4, 5]
    param_new["max_delta_step"] = [0.5, 0.6, 0.7, 0.8, 0.9] # best value found, default = 0
    param_new["scale_pos_weight"] = [2] # class imb
    
    print("--------end of parameter setting -------*")
    
    return param_new

In [13]:
iter = 0
while True:
    iter = iter + 1

    lgbm = None
    lgbmgs = None

    lgbm = lightgbm.LGBMClassifier(random_state=42)

    lgbmgs = GridSearchCV(estimator = lgbm,
                          param_grid = params_o,
                          cv = 5, # StratifiedKFold us default for binary or multiclass
                          scoring = 'precision', 
#                           scoring = 'accuracy', 
                          error_score='raise',
                          verbose = 1,
                          n_jobs=-1, # 자동 검색 적용
                          )

    lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss') # 

    df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), iter)
    result_dict = calc_results(lgbmgs.best_estimator_, train_scaled, val_scaled, test_scaled, test_target)
    
    df_result = make_df_from_estimator(result_dict, iter)
    df_concat = pd.concat([df_estimator, df_result])

    df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)
    
#  4가지 조건이 만족되면 break하고 완료
    val_test = df_concat.loc['acc_val'].iloc[0]
    acc_test = df_concat.loc['acc_test'].iloc[0]
    precision = df_concat.loc['precision'].iloc[0]
    f1score = df_concat.loc['f1score'].iloc[0]
    if (val_test >= 0.75 ) & (acc_test > 0.75) & (precision >= 0.8) & (f1score >= 0.6) :
        break
    if iter >= 7 : break
    print("*******Np.{}**Processing*********".format(iter))
    params_o = make_new_parameter(params_o, df_concat)

Fitting 5 folds for each of 36000 candidates, totalling 180000 fits


C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[LightGBM] [Warning] Unknown parameter: andom_state
********Processing*********
--------end of parameter setting -------*
Fitting 5 folds for each of 90000 candidates, totalling 450000 fits


C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: andom_state
********Processing*********
--------end of parameter setting -------*
Fitting 5 folds for each of 200000 candidates, totalling 1000000 fits


...

C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: andom_state
********Processing*********
--------end of parameter setting -------*
Fitting 5 folds for each of 150000 candidates, totalling 750000 fits


C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: andom_state
********Processing*********
--------end of parameter setting -------*
Fitting 5 folds for each of 150000 candidates, totalling 750000 fits


KeyboardInterrupt: 

In [14]:
# save the results
directory = './'
df_base.to_pickle(directory + 'df_mobis_lgbm.pkl')
df_base.to_csv(directory + 'df_mobis_lgbm.csv')

In [ ]:
## end

In [15]:
df_base

,value1,value2,value3,value4,value5,value6,value7,value8,value9,value10
parameter,,,,,,,,,,
boosting_type,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt
class_weight,None,None,None,None,None,None,None,None,None,None
colsample_bytree,0.25,0.45,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
importance_type,split,split,split,split,split,split,split,split,split,split
learning_rate,0.007,0.011,0.051,0.051,0.051,0.051,0.051,0.051,0.051,0.051
max_depth,None,None,2,2,2,2,2,2,2,2
min_child_samples,20,20,20,20,20,20,20,20,20,20
min_child_weight,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
min_split_gain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
params_o = make_new_parameter(params_o, df_concat)

In [ ]:
iter =0 
iter = iter + 1

lgbm = None
lgbmgs = None

lgbm = lightgbm.LGBMClassifier( random_state=42)

lgbmgs = GridSearchCV(estimator = lgbm,
                      param_grid = params_o,
                      cv = 5, # StratifiedKFold us default for binary or multiclass
#                       scoring = 'precision', 
                      scoring = 'accuracy', 
                      error_score='raise',
                      verbose = 1,
                      n_jobs=-1, # 자동 검색 적용
                      )

lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss') # 

df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), iter)
result_dict = calc_results(lgbmgs.best_estimator_, train_scaled, val_scaled, test_scaled, test_target)
df_result = make_df_from_estimator(result_dict, iter)
df_concat = pd.concat([df_estimator, df_result])

df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)

In [ ]:
print("Best Estimator: {}".format(lgbmgs.best_estimator_)) 
print("Best Parameters: {}".format(lgbmgs.best_params_))  # 최적 파라미터.
print('Best Score: {}, Best Index: {}'.format(lgbmgs.best_score_ , lgbmgs.best_index_))  # 교차검증된 점수를 보여줌.

model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
y_pred = model.predict(test_scaled)

train_score = model.score(train_scaled, train_target)
val_score = model.score(val_scaled, val_target)
test_score = model.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, model.predict(test_scaled))
cm = confusion_matrix(test_target, model.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score, val_score, test_score))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

In [17]:
model = lgbmgs.best_estimator_ 

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
model.feature_importances_

In [ ]:
df_sel = pd.DataFrame(model.feature_importances_, index=data.columns, columns=['importance']).sort_values(by='importance', ascending=False)

In [16]:
model.booster_.feature_importance(importance_type='gain')

NameError: name 'model' is not defined

In [ ]:
df_sel = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [ ]:
model.booster_.feature_importance(importance_type='split')
# 큰 특징을 가지는 feature는 tree상위레벨에서 적게 사용됨.

In [ ]:
# # 최초의 empty df 생성
# df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')

In [ ]:
df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), 1)
result_dict = calc_results(lgbmgs.best_estimator_, train_scaled, val_scaled, test_scaled, test_target)
df_result = make_df_from_estimator(result_dict, 1)
df_concat = pd.concat([df_estimator, df_result])

In [ ]:
df_concat

In [ ]:
df_concat.index =['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type',
       'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight',
       'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective',
       'random_state', 'reg_alpha', 'reg_lambda', 'silent', 'subsample',
       'subsample_for_bin', 'subsample_freq', 'max_delta_step', 'metric',
       'num_iterations', 'scale_pos_weight', 'best_score', 'best_index',
       'acc_train', 'acc_val', 'acc_test', 'precision', 'recall', 'f1score',
       'roc', 'tn', 'fp', 'fn', 'tp']
df_concat

In [ ]:
df_base

In [ ]:
# df_base = df_base.merge(df_estimator, how='outer', left_index=True, right_index=True)

In [ ]:
# df_base = df_base.merge(df_estimator, how='outer', left_index=True, right_index=True)

In [ ]:
# dfs = [df_estimator, df_result]
# df_merged= reduce(lambda  left,right: pd.merge(left,right, how='outer', left_index=True, right_index=True), dfs)

In [ ]:
df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)

In [ ]:
{'boosting_type': ['gbdt'],
 'num_iterations': [10, 50, 100],
 'learning_rate': [0.003, 0.005, 0.007, 0.009000000000000001, 0.011],
 'max_depth': [None, 1, 2, 3, 4],
 'num_leaves': [None, 1, 2, 3, 4],
 'subsample': [None, 0.001, 0.002, 0.003],
 'colsample_bytree': [0.25, 0.3, 0.35, 0.4, 0.45],
 'objective': ['binary'],
 'metric': ['binary_logloss'],
 'n_estimators': [None, 3, 4, 5],
 'max_delta_step': [0.5, 0.6, 0.7, 0.8, 0.9],
 'scale_pos_weight': [2]}


In [ ]:
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
    "num_iterations" : [5], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.007, 0.008, 0.009, 0.01, 0.015, 0.02], 
    "max_depth": [None, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [None, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [ 0.25, 0.3, 0.35, 0.4, 0.45], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [None, 3, 4, 5],
    "max_delta_step": [0.5, 0.6, 0.7, 0.8, 0.9], # best value found, default = 0
    "scale_pos_weight": [2], # cla

In [ ]:
pa = params_o

In [ ]:
params_o["boosting_type"] == pa["boosting_type"]

In [ ]:
param_new["max_delta_step"] = [0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
params_o["max_delta_step"]

In [ ]:
params_o["num_iterations"] = new_num_iter(50)

In [ ]:
params_o["num_iterations"]

In [ ]:
pp = make_new_parameter(params_o, df_concat)

In [ ]:
pp

In [ ]:
new_num_iter(20)

In [ ]:
aaa = {}
aaa['bb'] = new_num_iter(20)
aaa

In [ ]:
aaa['bb']

In [ ]:
aaa["boosting_type"] = ['gbdt']

In [ ]:
aaa

In [ ]:
param_new = {}
param_new["boosting_type"] = ['gbdt'], 
num_i = df_concat.loc["num_iterations"].iloc[0]
param_new["num_iterations"] = new_num_iter(num_i), #

In [ ]:
params_o

In [ ]:
lll = {}
lll["boosting_type"] = ['gbdt'], 

num_i = df_concat.loc["num_iterations"].iloc[0]

lll["num_iterations"] = new_num_iter(num_i), #

In [ ]:
lll